Input is a bigwig file and out comes a GRanges object. The 'score' metadata column is split into a T and M column, where the T column represents the total number of reads and has a value of 10 in all rows as we are assuming 10 total read coverage. The M column represents the methylation count at that point and is just the score column multiplied by 10 and floored to the nearest integer. Getting data from https://epigenomesportal.ca/ihec/grid.html?assembly=1&build=2018-10 

Instead of loading each pathname individually, I want to make it possible to extract each file from the folder individually. No clue how. 

In [ ]:
# About the data

To be clear, this whole processing step is faking observations that were used to calculate the original score or methylation percentage. This is a necessary step for the segmentPMDs function to work properly but it is technically faking data. We may have those observations in the LINA /LISA clinical data, but this is uncertain.


In [1]:
# Set wd JOKES U CAN'T HAHAHAHA

# Libraries take up ca. 715MB space
# DataFrames underlying GRanges
library(S4Vectors)

# Because the data samples I am using are from the IHEC 
#    https://epigenomesportal.ca/ihec/grid.html?assembly=1&build=2018-10 database as .bigwig:
# For importing .bigwig files: https://kasperdanielhansen.github.io/genbioconductor/html/rtracklayer_Import.html
library(rtracklayer)
library(AnnotationHub)
library(Rsamtools)

# To use mcols in GRanges
library(MethylSeekR)



Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min


Attaching package: ‘S4Vectors’

The following object is masked from ‘package:bas

# Downloading data
So we are going to automatically download by URL and thus keep track of which files we are downloading and where the data is coming from as it would be tedious to download them individually. Downloading all the links below is beyond the memory capacity of my laptop so I downloaded the top 19 URL's out of the 34 in the IHEC/hg19/DEEP/Blood/Tracks folder. The first 3 of those are *not* listed below.

The writeLines() function has syntax: writeLines(text, con = stdout(), sep = "\n", useBytes = FALSE)


In [3]:
# Specifying downloads / data to download
# Paste all url's you wanna download into this c() matrix with no spaces between them and /n separated
urlsToDownload = c("https://epigenomesportal.ca/tracks/DEEP/hg19/63217.DEEP.43_Hm05_BlMa_Ct.WGB-Seq.methylation_profile.bigWig
https://epigenomesportal.ca/tracks/DEEP/hg19/63216.DEEP.43_Hm05_BlMa_Ct.WGB-Seq.signal_unstranded.bigWig
https://epigenomesportal.ca/tracks/DEEP/hg19/63237.DEEP.43_Hm05_BlMa_TE.WGB-Seq.methylation_profile.bigWig
https://epigenomesportal.ca/tracks/DEEP/hg19/63236.DEEP.43_Hm05_BlMa_TE.WGB-Seq.signal_unstranded.bigWig
https://epigenomesportal.ca/tracks/DEEP/hg19/63257.DEEP.43_Hm05_BlMa_TO.WGB-Seq.methylation_profile.bigWig
https://epigenomesportal.ca/tracks/DEEP/hg19/63256.DEEP.43_Hm05_BlMa_TO.WGB-Seq.signal_unstranded.bigWig
https://epigenomesportal.ca/tracks/DEEP/hg19/63277.DEEP.43_Hm05_BlMo_Ct.WGB-Seq.methylation_profile.bigWig
https://epigenomesportal.ca/tracks/DEEP/hg19/63276.DEEP.43_Hm05_BlMo_Ct.WGB-Seq.signal_unstranded.bigWig
https://epigenomesportal.ca/tracks/DEEP/hg19/63297.DEEP.51_Hf01_BlCM_Ct.WGB-Seq.methylation_profile.bigWig
https://epigenomesportal.ca/tracks/DEEP/hg19/63296.DEEP.51_Hf01_BlCM_Ct.WGB-Seq.signal_unstranded.bigWig
https://epigenomesportal.ca/tracks/DEEP/hg19/63307.DEEP.51_Hf02_BlCM_Ct.WGB-Seq.methylation_profile.bigWig
https://epigenomesportal.ca/tracks/DEEP/hg19/63306.DEEP.51_Hf02_BlCM_Ct.WGB-Seq.signal_unstranded.bigWig
https://epigenomesportal.ca/tracks/DEEP/hg19/63318.DEEP.51_Hf02_BlEM_Ct.WGB-Seq.methylation_profile.bigWig
https://epigenomesportal.ca/tracks/DEEP/hg19/63317.DEEP.51_Hf02_BlEM_Ct.WGB-Seq.signal_unstranded.bigWig
https://epigenomesportal.ca/tracks/DEEP/hg19/63341.DEEP.51_Hf03_BlCM_Ct.WGB-Seq.methylation_profile.bigWig
https://epigenomesportal.ca/tracks/DEEP/hg19/63340.DEEP.51_Hf03_BlCM_Ct.WGB-Seq.signal_unstranded.bigWig
https://epigenomesportal.ca/tracks/DEEP/hg19/63350.DEEP.51_Hf03_BlEM_Ct.WGB-Seq.methylation_profile.bigWig
https://epigenomesportal.ca/tracks/DEEP/hg19/63349.DEEP.51_Hf03_BlEM_Ct.WGB-Seq.signal_unstranded.bigWig
https://epigenomesportal.ca/tracks/DEEP/hg19/63383.DEEP.51_Hf04_BlCM_Ct.WGB-Seq.methylation_profile.bigWig
https://epigenomesportal.ca/tracks/DEEP/hg19/63382.DEEP.51_Hf04_BlCM_Ct.WGB-Seq.signal_unstranded.bigWig
https://epigenomesportal.ca/tracks/DEEP/hg19/63406.DEEP.51_Hf04_BlEM_Ct.WGB-Seq.methylation_profile.bigWig
https://epigenomesportal.ca/tracks/DEEP/hg19/63408.DEEP.51_Hf04_BlEM_Ct.WGB-Seq.methylation_profile.bigWig
https://epigenomesportal.ca/tracks/DEEP/hg19/63405.DEEP.51_Hf04_BlEM_Ct.WGB-Seq.signal_unstranded.bigWig
https://epigenomesportal.ca/tracks/DEEP/hg19/63407.DEEP.51_Hf04_BlEM_Ct.WGB-Seq.signal_unstranded.bigWig")

urlsFile <- file('Desktop/Kode/Bioinf_Internship/urlsToDownload.txt')    # creating file name urlsToDownload.txt in the working directory /Users/oliviagallup/Desktop/Kode/
writeLines(urlsToDownload, urlsFile)    # writes the list of our url's to the txt file we just created
close(urlsFile)

# Too lazy to find how to download URL's using R; do 
# wget --no-check-certificate -P /Users/oliviagallup/Downloads/IHEC_blood_cells -i /Users/oliviagallup/Desktop/Kode/urlsToDownload.txt


# Loading Data

To make it EasyTM to download data, paste all the url's that lead directly to your download into the urlsToDownload list, which get put into a txt document for wget to read in command line    wget --no-check-certificate -P /Users/oliviagallup/Downloads/IHEC_blood_cells -i /Users/oliviagallup/Desktop/Kode/urlsToDownload.txt    where the -i tells wget to read the url's out of the txt file, the -P sets the directory the files will be downloaded to and the --no-check-certificate means wget doesn't check the certificate fo the site it's downloading from, which I have to put because apparently the IHEC website is too untrustworthy for wget to download from.

This is because with the download.file(url, destfile, method = "auto", quiet=FALSE) function in R you have to specify a destination file and that's just too hard.

Because our files downloaded to their own folder, it will not be hard to load them and process the GRanges objects properly, which is what we need to do to the bigwig files (or any other file type) so that the segmentPMDs function can work with the objects. The rtracklayer::import() function only takes in one file at a time and turns it into a GRanges object, so we need to cycle through all the files we downloaded. Will use the list.files() R funciton to get path names. See https://stackoverflow.com/questions/16566799/change-variable-name-in-for-loop-using-r and https://stat.ethz.ch/R-manual/R-patched/library/base/html/assign.html for iteratively creating variable names. 

The type of cells are mixed and intelligible from the filename with certain keywords. Check the IHEC Data portal https://epigenomesportal.ca/ihec/grid.html?assembly=1&build=2018-10 to isolate the cell type of the samples by filename. Because we want to demo a comparison of different cell types, I'm going to use three samples that are monocytes and three that are central memory CD4-positive, alpha-beta T cell. And leave out the samples marked with 'TE' and 'TO', only those marked 'Ct'.

For different cell types: To specify that we want files with a certain word in the name (ie 'BlMo' for monocytes) we can input that into the pattern attribute of list.files().

In [2]:
# Retrieving data file pathnames
# Do NOT USE THE FULL LIST OF DATA - the program will take up loads of memory to manage that shit >16GB;
# use the subset of the pathnames. Three files is equivalent to like 5GB lmao
pathnames = list.files(path = "/Users/oliviagallup/Downloads/IHEC_blood_cells/Methylomes", pattern="BlMo", full.names = TRUE)

start <- 1
finish <- 3  # Choosing the files to start and to finish reading in on. We need this for the variable naming

pathnames_subset <- pathnames[start:finish]   # for loop testing purposes
print(pathnames_subset)

granges.env <- new.env()    # Environment that will contain all the GRanges variables

# names_list <- c(0)    # UNNECESSARY AFTER ALL; Saves all the names of the variables we are using; might be unnecessary 
file_count <- start    # Counter for naming the variables

# Loading data using import function from rtracklayer
for (i in pathnames_subset) {
    nam <- paste("importedBigWig_monocytes_", file_count, sep = "")     # iteratively create GRanges object variables, using 
#     paste and assign methods, end up with importedBigWig_1, importedBigWig_2, etc.
    assign(nam, import(format="BigWig", con=i), envir=granges.env)
    file_count <- file_count+1
#     append(names_list, nam)        FOR SOME REASON APPEND DOESN'T WORK
}


[1] "/Users/oliviagallup/Downloads/IHEC_blood_cells/Methylomes/63119.DEEP.43_Hm01_BlMo_Ct.WGB-Seq.methylation_profile.bigWig"
[2] "/Users/oliviagallup/Downloads/IHEC_blood_cells/Methylomes/63195.DEEP.43_Hm03_BlMo_Ct.WGB-Seq.methylation_profile.bigWig"
[3] "/Users/oliviagallup/Downloads/IHEC_blood_cells/Methylomes/63277.DEEP.43_Hm05_BlMo_Ct.WGB-Seq.methylation_profile.bigWig"


Because our files are so massive, we will have to read them in a couple at a time, then save them as compressed RData files and delete the ones we just processed. To delete processed files we can use the file names in pathnames_subset and copy paste that into a command line remove function like $ rm filename1 filename2 filename3. 

The data we load in has variable names created on the fly, like data_1, data_2, etc. We will keep these variables nice and packed up in their own environment. In the assign() function you can state what environment the var's are to be created in. We can then get the variables from that environment and manipulate them to have T and M columns, then save them.

12/08/2019: I'm acc so smart
To access the data we extracted into the dummy environment, all you gotta do is save all the objects from the dummy env that you return with the list method, then use the multi get method (though mget acc stands for more get) to make a list of GRanges objects helly eah. With the _lapply(x, fun)_ function we can apply a function to each element within x. This means a function that turns the methylation 'scores' columns into M and T we wouldn't need to use a for loop, which in R is over 10x slower than the lapply function.

About environments: https://stat.ethz.ch/R-manual/R-devel/library/base/html/environment.html
About getting/setting in env's: https://stat.ethz.ch/R-manual/R-devel/library/base/html/get.html

In [3]:
granges_names <- ls(granges.env)

granges_list <- mget(granges_names, envir=granges.env)
# granges_list

The processGRanages() function below will serve as the function input to the lapply() loop we want to use on all the GRanges that we have loaded, allowing us to process them for use by the segmentPMDs function. The mcols() function accesses the metadata column in the GRanges object and fuxx with it. Unfortunately not all the 'scores' in the bigwig files is in a 0 to 1 range; some are already in a 0-100 range. This complicates things, since there might be a score <1 but that's acc on a 0-100 range. Need to handle the different ranges; first get the largest score in the 'scores' mcol, then use that to handle the two different cases. Regardless, we need to bring the M column to a 0-10 scale. 

To be clear, this whole processing step is faking observations that were used to calculate the original score or methylation percentage. This is a necessary step for the segmentPMDs function to work properly but it is technically faking data. We may have those observations in the LINA /LISA clinical data, but this is uncertain.

Although this is an arbitrarily decided on scale to bring the values to and does not afford as high a resolution as if we brought the scores to the 0-1000 scale, this is on purpose because our read count is only around 15-20. So to be extra conservative and pessimistic, we assume a max of 10 reads per base, hence the need to bring the integers to a 0-10 scale.


In [4]:
# Function for converting 'scores' to M and T 

processGRanges <- function(GRange) {
    rangeMax <- max(mcols(GRange)[,1])       # Figure out what range the 'score' was created in, 0-1 or 0-100
    
    M <- matrix(0, length(GRange), 1)        # Initializing the M column with trash values of 0 and dimensions of the data
    
    if (rangeMax >= 1) {
        # Case where the data was scored on a 0-100 scale
        M[,1] <- floor((mcols(GRange)[, 1]) %/% 10)        # Loop through all the metadata values in the 'score' column, 
#     multiply them by 10, assign to M (we don't even need a for loop for this)
    } else {
        # Case where the data was scored on a 0-1 scale        
        M[,1] <- floor((mcols(GRange)[, 1])*10)        # Loop through all the metadata values in the 'score' column, 
#     multiply them by 10, assign to M (we don't even need a for loop for this)
    }
    
    
    M <- pmax(0, M)       # Turn all negative bois to 0 #relu using 'parallel max' fxn
    
    mcols(GRange)[, 1] <- matrix(10, length(GRange), 1)     # T column all 10
    mcols(GRange)[, 2] <- M 
    
    # Renaming metadata columns
    names(mcols(GRange))[1] <- 'T'
    names(mcols(GRange))[2] <- 'M'
    return(GRange)
}



# Processing and saving the Data

It took me 2 hours to realize that the max() function wasn't working because I was feeding it the whole data-frame of the metadata instead of just a single column. The function first finds the highest methylation score in the 'score' column, then initiates an M vector full of 0's, then decides how to fake the T and M column observations based on the highest methylation score we found, which tells us the range that was used to record mehtylation.

Now to save the data. The names() function returns the name of the []th element in the list that is put into the function, which is an accessor to the 'names' attribute in the list that has the name of each key stored. Found by https://stackoverflow.com/questions/10198544/r-get-key-from-key-value-hash

In [5]:
# Saving all the data into an R data file, which is very compressed and will allow us to
# store twice the amount of data


granges_list <- lapply(granges_list, processGRanges)     # Fast loop that applies the function to each element in list

for(i in 1:length(granges_list)) {
    saveRDS(granges_list[[i]], file=names(granges_list)[i])
}


Saving and processing seem to work, to double check everything went right we can read in one of the files and see what the output looks like. We can also print the path names of the files we just converted to GRanges so that we can copy them and remove them using   rm /pathname/file in command line. We might also want to keep track of what files we've already downloaded; write the names to a text file. To specify that we want files with a certain word in the name (ie 'methylation') we can input that into the pattern attribute of list.files.

In [6]:

filesProcessed <- file('filesProcessed.txt')    # creating file name filesProcessed.txt in the working directory /Users/oliviagallup/Desktop/Kode/
writeLines(pathnames_subset, filesProcessed)    # writes the files we processed / want to delete to the txt file
close(filesProcessed)

In [53]:
readRDS(names(granges_list)[1])

GRanges object with 48661238 ranges and 2 metadata columns:
             seqnames    ranges strand |         T         M
                <Rle> <IRanges>  <Rle> | <numeric> <numeric>
         [1]     chr1     10470      * |        10         0
         [2]     chr1     10472      * |        10         0
         [3]     chr1     10485      * |        10         0
         [4]     chr1     10490      * |        10         0
         [5]     chr1     10494      * |        10         0
         ...      ...       ...    ... .       ...       ...
  [48661234]     chrX 155254994      * |        10         0
  [48661235]     chrX 155255018      * |        10         0
  [48661236]     chrX 155255019      * |        10         0
  [48661237]     chrX 155255024      * |        10         0
  [48661238]     chrX 155255025      * |        10         0
  -------
  seqinfo: 23 sequences from an unspecified genome